<a href="https://colab.research.google.com/github/broadinstitute/palantir-workflows/blob/mg_benchmark_compare/BenchmarkVCFs/ExportToGoogleSheets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Export comparison data to Google Sheet

This notebook will extract the comparison data generated with the `CompareBenchmarks` workflow and export it as a new sheet (tab) to a Google spreadsheet, including the required (conditional) formatting.

__*Note*__: The Google authentication process requires this notebook to be run in [Google Colab](https://colab.research.google.com/) using the link above.

## Install the Firecloud API

This is required in case the [Firecloud API](https://github.com/broadinstitute/fiss) isn't installed in the current runtime.

In [ ]:
try:
    import firecloud.api as fapi
except ModuleNotFoundError:
    %pip install firecloud

## Data export

In the cell below, enter the `gs://` path to the `comparison.csv` file, which is the output of the `CompareBenchmarks` workflow.

In addition, please enter the Google Spreadsheet ID of the notebook that you want to export the data to. The spreadsheet ID is shown in the URL in the browser. If you want to create a new spreadsheet you can do so on [sheets.google.com](https://sheets.google.com) or simply navigate to [sheet.new](https://sheet.new).

The notebook will automatically create a new sheet (tab) in the provided spreadsheet, named with the current date and time (in UTC).

__*Note*__: When executing the cell you will be provided with a link to authenticate this notebook to access your Google sheets. Please click that link, authenticate with Google, and enter the resulting code into the prompt in this notebook.

In [ ]:
comparison_csv_gs_path = "gs://fc-640bf3c9-3bb5-45c1-a59f-a271111337da/683fd921-ab36-4382-8aa7-eb5d27716469/CompareBenchmarks/4fff91a6-e2d1-442f-869c-92134b84d092/call-CompareBenchmarksTask/output.csv" #@param {type:"string"}

spreadsheet_id = "1kVu86hxxdsHe7bxXpKq5cIcPA29v8rQl9RU5TEss8-Q" #@param {type:"string"}

import firecloud.api as fapi
import datetime
from google.colab import auth
auth.authenticate_user()
from googleapiclient.discovery import build

service = build('sheets', 'v4')

csv_lines = !gsutil cat {comparison_csv_gs_path}
csv = '\n'.join(csv_lines)

new_sheet_name = f'comparison{datetime.datetime.now()}'

new_sheet_body = {
        'requests': [
            {
                'addSheet': {
                    'properties': {
                        'title': new_sheet_name
                    }
                }
            }
        ]
    }

new_sheet_request = service.spreadsheets().batchUpdate(spreadsheetId=spreadsheet_id, body=new_sheet_body)
new_sheet_response = new_sheet_request.execute()
new_sheet_id = new_sheet_response['replies'][0]['addSheet']['properties']['sheetId']

body = {
        'requests': [{
            'pasteData': {
                "coordinate": {
                    "sheetId": new_sheet_id,
                    "rowIndex": "0",  # adapt this if you need different positioning
                    "columnIndex": "0", # adapt this if you need different positioning
                },
                "data": csv,
                "type": 'PASTE_NORMAL',
                "delimiter": ',',
            }
        }]
    }
csv_import_request = service.spreadsheets().batchUpdate(spreadsheetId=spreadsheet_id, body=body)
csv_import_response = csv_import_request.execute()

rows_request = service.spreadsheets().values().get(spreadsheetId=spreadsheet_id, range=f'{new_sheet_name}!1:3')
rows_response = rows_request.execute()

num_metrics = 3 if rows_response['values'][0][4] == 'Precision' else 6
num_samples = len(set([item for item in rows_response['values'][1] if item]))
num_configurations = len(set([item for item in rows_response['values'][2] if item and not item.startswith('delta%(') and not item.startswith('delta(')]))
num_deltas = len(set([item for item in rows_response['values'][2] if item and item.startswith('delta%(')]))

cols_request = service.spreadsheets().values().get(spreadsheetId=spreadsheet_id, range=f'{new_sheet_name}!C:C')
cols_response = cols_request.execute()

num_stratifiers = len(set([item for sublist in cols_response['values'] for item in sublist if item]))

def add_box(row_from, row_to, col_from, col_to):
    return {
      "updateBorders": {
        "range": {
          "sheetId": new_sheet_id,
          "startRowIndex": row_from,
          "endRowIndex": row_to,
          "startColumnIndex": col_from,
          "endColumnIndex": col_to
        },
        "top": {
          "style": "SOLID"
        },
        "bottom": {
          "style": "SOLID"
        },
        "left": {
          "style": "SOLID"
        },
        "right": {
          "style": "SOLID"
        },
        }}
def merge_cells(row_from, row_to, col_from, col_to):
    return {
      "mergeCells": {
        "range": {
          "sheetId": new_sheet_id,
          "startRowIndex": row_from,
          "endRowIndex": row_to,
          "startColumnIndex": col_from,
          "endColumnIndex": col_to
        },
        "mergeType": "MERGE_ALL"
      }
    }

def center_cells_row(row_from, row_to):
    return {"repeatCell": {
        "range": {
          "sheetId": new_sheet_id,
          "startRowIndex": row_from,
          "endRowIndex": row_to
        },
        "cell": {
          "userEnteredFormat": {
            "horizontalAlignment" : "CENTER",
            "verticalAlignment" : "MIDDLE",
            }
          },
        "fields": "userEnteredFormat(horizontalAlignment, verticalAlignment)"
      },
    }

def center_cells_col(col_from, col_to):
    return {"repeatCell": {
        "range": {
          "sheetId": new_sheet_id,
          "startColumnIndex": col_from,
          "endColumnIndex": col_to
        },
        "cell": {
          "userEnteredFormat": {
            "horizontalAlignment" : "CENTER",
            "verticalAlignment" : "MIDDLE",
            }
          },
        "fields": "userEnteredFormat(horizontalAlignment, verticalAlignment)"
      },
    }

def conditional_formatting(row_from, row_to, col_from, col_to):
    return {
      "addConditionalFormatRule": {
        "rule": {
          "ranges": [
            {
              "sheetId": new_sheet_id,
              "startRowIndex": row_from,
              "endRowIndex": row_to,
              "startColumnIndex": col_from,
              "endColumnIndex": col_to
            }
          ],
          "gradientRule": {
            "minpoint": {
              "color": {
                "red": 0.8984375,
                "green": 0.484375,
                "blue": 0.44921875
              },
              "type": "NUMBER",
              "value": "-0.01"
            },
            "midpoint": {
              "color": {
                "red": 1,
                "green": 1,
                "blue": 1
              },
              "type": "NUMBER",
              "value": "0"
            },
            "maxpoint": {
              "color": {
                "red": 0.33984375,
                "green": 0.73046875,
                "blue": 0.5390625
              },
              "type": "NUMBER",
              "value": "0.01"
            },
          }
        },
        "index": 1
      }
    }

requests = []

# Merge header
base = 4
for i_metric in range(num_metrics):
    base_metric = base + i_metric * (num_samples * (num_configurations + num_deltas))
    requests.append(merge_cells(0, 1, base_metric, base_metric + num_samples * (num_configurations + num_deltas)))
    requests.append(add_box(0, 1, base_metric, base_metric + num_samples * (num_configurations + num_deltas)))

    for i_sample in range(num_samples):
        base_sample = base_metric + i_sample * (num_configurations + num_deltas)
        requests.append(merge_cells(1, 2, base_sample, base_sample + num_configurations + num_deltas))
        # Add sample border
        requests.append(add_box(1, 3 + 3 * num_stratifiers, base_sample, base_sample + num_configurations + num_deltas))

        # Add conditional background
        requests.append(conditional_formatting(2, 3 + 3 * num_stratifiers, base_sample + num_configurations, base_sample + num_configurations + num_deltas))

# Merge stratifiers
base = 3
for i_stratifier in range(num_stratifiers):
    base_stratifier = base + i_stratifier * 3
    requests.append(merge_cells(base_stratifier, base_stratifier + 3, 2, 3))
    requests.append(merge_cells(base_stratifier, base_stratifier + 3, 2 + 2 + num_metrics * num_samples * (num_configurations + num_deltas) + 1, 2 + 2 + num_metrics * num_samples * (num_configurations + num_deltas) + 2))  
# Border stratifiers
base = 3
for i_stratifier in range(num_stratifiers):
    base_stratifier = base + i_stratifier * 3
    requests.append(add_box(base_stratifier, base_stratifier + 3, 2, 2 + 2 + num_metrics * num_samples * (num_configurations + num_deltas) + 2))
  
requests.append(center_cells_row(0, 2))
requests.append(center_cells_col(2, 4))
requests.append(center_cells_col(2 + 2 + num_metrics * num_samples * (num_configurations + num_deltas), 2 + 2 + num_metrics * num_samples * (num_configurations + num_deltas) + 2))

body = {
        'requests': requests
    }
border_request = service.spreadsheets().batchUpdate(spreadsheetId=spreadsheet_id, body=body)
border_response = border_request.execute()